### Importing The Required Libraries:

In [1]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer

tf.__version__
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


import pickle
import numpy as np

import os
import os.path

import string
from itertools import chain

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 2080 SUPER, pci bus id: 0000:05:00.0, compute capability: 7.5



In [2]:
filename = "qa-1M"
fname = "lazar"
tokenizer = pickle.load(open('tokenizer/tokenizer-' + filename + '.pkl', 'rb'))


In [3]:
file = open("data/" + fname + ".txt", "r", encoding = "utf8")
lines = []
l = 0
for i in file:
    lines.append(i)
    l = l + 1
if (l > 5):
    print("The First Line: ", lines[0])
    print("The Last Line: ", lines[-1])
else:
    print(len(lines[0]))
data = ""

for i in lines:
    data = ' '. join(lines)

data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '')

data[:360]

The First Line:   No, like I said, I'm not going to really concern myself with that. Not till later.

The Last Line:   From his mom...


" No, like I said, I'm not going to really concern myself with that. Not till later.  I wasn't told what I was working on...  At that point... no.  Yeah.  The best thing for me to do is to pull out...  I have these calendars... they're big wall calendars where I write what happened every day on it... and I've had them since like 1980... so I have exact dates."

In [14]:
sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:10]


[1607, 35, 867, 1551, 10, 1018, 54, 60, 9103, 8837]

In [15]:

vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

10457


In [5]:
def defineData():
    sequences = []
    for i in range(1, int(len(sequence_data))):
        words = sequence_data[i-1:i+1]
        sequences.append(words)

    #print("The Length of sequences are: ", len(sequences))
    sequences = np.array(sequences)
    #sequences[:10]


    X = []
    y = []

    for i in sequences:
        X.append(i[0])
        y.append(i[1])

    X = np.array(X)
    y = np.array(y)

    y = to_categorical(y, num_classes=vocab_size)
    #y[:5]
    return X,y

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import load_model


model = load_model('nextword-'+filename+'.h5')

checkpoint = ModelCheckpoint("nextword-"+filename+".h5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto')

reduce = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)

logdir='logsnextword'+filename
tensorboard_Visualization = TensorBoard(log_dir=logdir)

model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))

In [16]:
X,y = defineData()

print("The Data is: ", X[:5])
print("The responses are: ", y[:5])


The Data is:  [1607   35  867 1551   10]
The responses are:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
model.fit(X, y, epochs=50, batch_size=64, callbacks=[checkpoint, reduce, tensorboard_Visualization])


In [13]:
print(X , len(X))

[1607   35  867 ...  257 1475  559] 36078


In [9]:
def Predict_Next_Words(model, tokenizer, text):
    #print(text)
    sequence = tokenizer.texts_to_sequences([text])[0]
    sequence = np.array(sequence)
    res = []
    preds = model.predict(sequence)
    preds1= np.flip(np.argsort(preds,axis=1))
    preds = np.argmax(preds,axis=1)
    for i in range(10):
        res.append((preds1[0][i]))
    #print(len(res))
    predicted_word = ""
    predicted_words = []

    for key, value in tokenizer.word_index.items():
        if value in res:
            predicted_words.append(key)
            #print(key)
            #break

    #print(len(predicted_word))
    return predicted_words

In [10]:

text = "Obviously";
export = Predict_Next_Words(model, tokenizer, text)
print(export)

['not', 'it', 'well', 'an', 'when', 'at', 'this', 'oh', 'she', 'cause']


In [11]:
strs = ["All","And","Boy","Book","Call","Car","Chair","Children","City","Dog","Door","Enemy","End","Enough","Eat","Friend","Father","Go","Good","Girl","Food","Hear","House","Inside","Laugh","Listen","Man","Name","Never","Next","New","Noise","Often","Pair","Pick","Play","Room","See","Sell","Sit","Speak","Smile","Sister","Think","Then","Walk","Water","Work","Write","Woman","Yes"]

for i in range(50):
    export = Predict_Next_Words(model, tokenizer, strs[i])
    print(strs[i] + "\t" + "\t".join(export))

All	the	that	it	there	this	i	these	just	of	kinds
And	the	that	they	it	there	so	then	i	we	he
Boy	and	that	you	when	i	odd	yeah	probably	joke	wierd
Book	and	or	there	was	more	together	where	had	we	over
Call	and	the	that	it	to	but	from	him	them	4
Car	and	that	in	they	is	to	was	back	go	i
Chair	that	or	out	so	down	which	with	me	from	of
Children	and	the	it	like	so	on	i	yeah	for	a
City	and	so	materials	oh	from	of	ah	maintenance	truck	wierd
Dog	and	that	is	so	as	with	i	instead	station	thing
Door	and	the	they	it	to	so	but	on	i	yeah
Enemy	and	that	so	on	i	from	who	station	thing	total
End	and	that	or	to	well	i	after	of	ironically	no
Enough	and	the	it	to	but	at	on	i	yeah	a
Eat	and	that	in	us	well	i	just	see	right	a
Friend	that	it	or	out	but	more	at	going	who	of
Father	i	yeah	much	edge	essentially	reasons	wrote	died	speakers	compositions
Go	and	out	to	through	back	on	down	up	ahead	into
Good	and	in	well	glad	point	friend	impression	analogy	job	portion
Girl	what	and	that	in	you	so	well	but	on	of
Food	